## Nearest Neighbors

***

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
# import shap
# import statsmodels.api as sm
# import datetime
# from datetime import datetime, timedelta
# import scipy.stats
# import pandas_profiling
# from pandas_profiling import ProfileReport
# import graphviz

# import xgboost as xgb
# from xgboost import XGBClassifier, XGBRegressor
# from xgboost import to_graphviz, plot_importance

#from sklearn.experimental import enable_hist_gradient_boosting
#from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, LogisticRegression, Ridge
#from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesClassifier, ExtraTreesRegressor
#from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, HistGradientBoostingClassifier, HistGradientBoostingRegressor


%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)


#from sklearn.pipeline import Pipeline
#from sklearn.model_selection import RepeatedStratifiedKFold
#from sklearn.feature_selection import RFE, RFECV, SelectKBest, f_classif, f_regression, chi2

from sklearn.neighbors import KNeighborsClassifier

from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import export_graphviz, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_squared_error,r2_score
from sklearn.metrics import plot_confusion_matrix, plot_precision_recall_curve, plot_roc_curve, accuracy_score
from sklearn.metrics import auc, f1_score, precision_score, recall_score, roc_auc_score


#from tpot import TPOTClassifier, TPOTRegressor
#from imblearn.under_sampling import RandomUnderSampler
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

# import pickle
# from pickle import dump, load

# Use Folium library to plot values on a map.
#import folium

# Use Feature-Engine library

#import feature_engine.missing_data_imputers as mdi
#from feature_engine.outlier_removers import Winsorizer
#from feature_engine import categorical_encoders as ce


np.random.seed(0)

#from pycaret.classification import *
#from pycaret.clustering import *
#from pycaret.regression import *

pd.set_option('display.max_columns',100)
#pd.set_option('display.max_rows',100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


Previously, we used logistic regression to predict sex based on height. Now we are going to use knn to do the same. Set the seed to 1, then use the caret package to partition the dslabs heights data into a training and test set of equal size. Use the sapply() function to perform knn with k values of seq(1, 101, 3) and calculate F1 scores with the F_meas() function using the default value of the relevant argument.

In [2]:
df = pd.read_csv("heights.csv")

In [3]:
df

,sex,height
0,Male,75.00
1,Male,70.00
2,Male,68.00
3,Male,74.00
4,Male,61.00
...,...,...
1045,Female,69.00
1046,Male,69.00
1047,Male,63.39
1048,Male,66.00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sex     1050 non-null   object 
 1   height  1050 non-null   float64
dtypes: float64(1), object(1)
memory usage: 16.5+ KB


## Data Preprocessing

### Replacing values

In [5]:
df["sex"] = df["sex"].apply(lambda x: 0 if x=="Male" else 1)

In [6]:
df

,sex,height
0,0,75.00
1,0,70.00
2,0,68.00
3,0,74.00
4,0,61.00
...,...,...
1045,1,69.00
1046,0,69.00
1047,0,63.39
1048,0,66.00


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sex     1050 non-null   int64  
 1   height  1050 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 16.5 KB


### Train Test Split

In [8]:
X = df.height
y = df.sex

In [9]:
X.values.reshape(-1,1), y.values

(array([[75.        ],
        [70.        ],
        [68.        ],
        ...,
        [63.38582677],
        [66.        ],
        [66.        ]]),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X.values.reshape(-1,1), y.values, test_size=0.5, random_state=0)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((525, 1), (525, 1), (525,), (525,))

### Using Regression or Classification Models

In [12]:
knn_r_acc = []

for i in range(1,103,3):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    F1score = f1_score(y_test,y_pred)
    knn_r_acc.append((i,F1score))
    


What is the max value of F_1?

At what value of k does the max occur?

In [13]:
result = pd.DataFrame(knn_r_acc, columns=['K','F1 Score'])
result

,K,F1 Score
0,1,0.50
1,4,0.44
2,7,0.56
3,10,0.56
4,13,0.55
5,16,0.57
6,19,0.62
7,22,0.62
8,25,0.64
9,28,0.62


Next we will use the same gene expression example used in the Comprehension Check: Distance exercises. You can load it like this:

In [14]:
df2 = pd.read_csv("tissue.csv")

In [15]:
df2.head()

,x.MAML1,x.LHPP,x.SEPT10,x.B3GNT4,x.ZNF280D,x.SOX12,x.C21orf62,x.PER3,x.HOXA10,x.HOXC5,x.BLVRB,x.ZIM2,x.HEMK1,x.FAP,x.MAN1A1,x.CDA,x.HTR7P1,x.DALRD3,x.FIBP,x.TTTY15,x.SLC30A1,x.SHANK2,x.MSL2,x.UBOX5,x.DUSP13,x.GJB5,x.MTF2,x.PPP1CA,x.IGHMBP2,x.VEGFA,x.KANSL1L,x.FCN3,x.USP32P2,x.HIVEP3,x.HRH1,x.HDAC7,x.HTT,x.IDH3A,x.TLR3,x.F11R,x.MOAP1,x.ISOC2,x.CLIP3,x.FZD10,x.VOPP1,x.RPL4,x.NUDT2,x.RAB30,x.DBI,x.CCDC87,...,x.ZNHIT3,x.DBF4B,x.GLUD1,x.ADRB2,x.FBXW4,x.SPP1,x.F2R,x.SDF4,x.GALNT8,x.TMEM63A,x.BSCL2,x.GZMM,x.THSD4,x.PRLR,x.PLEKHJ1,x.TLE3,x.PANK2,x.SKP1,x.TOMM70,x.KIAA1324,x.PCDHB12,x.CPA4,x.TRPV2,x.CHCHD2,x.TRPC6,x.MYO1D,x.SLURP1,x.ALG9,x.ZC4H2,x.KIR2DL3,x.IVL,x.BAMBI,x.SLC7A6,x.SLC17A1,x.CCL3,x.PHF8,x.KIR3DL3,x.SARS2,x.PIP4K2C,x.S100A13,x.EPHA1,x.MFGE8,x.OAZ2,x.PCBP3,x.POLA1,x.KREMEN2,x.CYP7B1,x.LILRB3,x.GSAP,y
0,9.83,8.33,5.50,8.69,5.64,6.25,5.84,8.33,5.52,7.66,7.67,8.36,7.34,6.26,5.33,6.19,5.12,6.34,9.63,5.71,7.82,8.06,7.73,8.89,7.20,6.63,9.62,8.66,7.11,8.74,5.46,6.63,9.96,7.93,5.91,8.10,8.39,7.92,5.22,7.70,10.56,8.03,11.72,5.97,9.40,11.96,7.27,6.87,10.73,6.75,...,9.00,5.77,8.48,7.04,8.61,11.07,4.63,10.22,6.43,6.76,9.95,6.44,5.92,4.62,8.45,5.82,8.72,11.46,7.41,6.17,7.33,8.57,6.93,11.70,4.96,5.09,6.52,6.29,8.69,7.27,7.26,6.49,5.59,6.96,6.21,7.37,7.27,7.41,8.99,8.64,7.02,8.44,10.23,8.46,7.35,8.39,4.76,8.23,6.74,cerebellum
1,9.63,8.54,5.64,8.83,5.69,6.29,6.07,8.26,5.53,7.57,7.78,8.65,7.28,6.38,5.27,6.20,5.15,6.29,9.94,6.41,8.43,8.34,8.10,8.98,7.14,6.46,9.57,9.05,7.18,9.10,5.41,6.47,10.06,7.93,6.15,8.20,8.18,7.78,5.26,7.78,10.55,8.21,12.05,6.09,9.67,11.75,7.13,6.90,11.55,6.87,...,8.74,5.94,9.03,7.23,8.60,7.01,4.70,10.22,6.51,6.75,10.14,6.63,6.34,4.58,8.64,6.04,8.76,11.30,6.83,6.09,7.35,8.57,6.99,11.85,5.06,5.08,6.77,6.46,8.68,7.30,7.36,6.92,5.60,6.86,7.12,7.35,7.43,7.68,9.21,9.18,7.10,8.63,9.84,8.78,7.48,8.26,4.95,8.33,6.39,cerebellum
2,9.69,8.48,5.72,8.50,5.96,6.22,5.77,9.10,5.60,7.42,7.55,8.85,7.24,6.12,5.51,5.99,4.98,6.27,9.81,6.53,8.27,8.58,7.72,8.64,7.08,6.47,9.67,8.64,7.18,8.12,5.54,6.59,10.65,7.89,5.83,8.05,8.14,8.87,5.19,7.40,11.05,7.80,11.79,5.88,9.90,11.82,7.09,6.83,11.43,6.87,...,9.19,5.73,8.67,6.97,8.67,8.66,4.56,10.18,6.33,6.84,10.24,6.53,6.01,4.64,8.59,5.93,8.76,11.61,7.55,5.98,7.08,8.42,6.84,11.68,4.96,5.12,6.44,6.36,8.53,7.14,7.10,7.00,5.54,6.85,5.94,7.30,7.48,7.33,9.27,8.88,6.74,8.19,10.06,8.52,7.30,8.28,4.80,7.98,6.68,cerebellum
3,9.99,8.51,5.79,8.42,5.74,6.51,5.85,9.21,5.56,7.33,7.84,8.20,7.21,6.28,5.33,5.82,4.86,6.64,9.75,6.44,8.62,8.59,7.92,8.69,6.99,6.37,9.48,8.81,7.24,8.82,5.47,6.65,10.29,7.72,5.79,8.36,8.22,8.01,5.28,7.42,10.71,7.98,12.00,6.01,9.85,11.88,7.18,6.58,12.04,6.65,...,9.06,5.62,8.94,6.85,8.89,8.10,4.56,10.43,6.36,6.56,10.27,6.34,6.00,4.67,8.40,5.71,8.80,11.44,7.11,5.84,7.17,8.18,6.72,11.72,4.93,5.15,6.28,6.54,8.25,7.06,7.06,6.80,5.48,6.74,6.04,7.38,7.18,7.31,9.02,8.71,6.85,8.28,9.91,8.21,7.45,8.28,4.96,7.89,6.82,cerebellum
4,9.58,8.37,5.78,8.82,5.63,6.20,5.97,8.50,5.60,7.54,7.60,8.58,7.53,6.34,5.33,6.02,5.11,6.12,9.58,5.79,8.30,8.61,7.55,8.83,7.10,6.26,9.52,8.61,7.17,9.14,5.40,6.73,9.79,7.86,5.91,7.80,8.34,7.99,5.23,7.25,10.72,7.96,11.78,5.73,9.33,11.93,7.20,6.77,11.44,6.70,...,8.91,5.44,8.84,6.79,8.64,9.43,4.70,10.09,6.35,6.83,10.11,6.68,6.25,4.57,8.66,5.81,8.70,11.30,7.53,5.99,7.26,8.80,6.65,11.46,5.07,5.14,6.59,6.32,8.72,7.37,7.47,6.54,5.53,6.92,6.10,7.24,7.25,7.56,9.23,8.85,6.84,8.04,10.03,8.76,7.25,8.47,4.87,8.13,7.00,cerebellum


In [16]:
df2.shape

(189, 501)

In [17]:
df2.y.value_counts()

kidney         39
cerebellum     38
colon          34
hippocampus    31
liver          26
endometrium    15
placenta        6
Name: y, dtype: int64

In [18]:
le = LabelEncoder()

In [19]:
df2["y"] = le.fit_transform(df2.y)

In [20]:
df2.head()

,x.MAML1,x.LHPP,x.SEPT10,x.B3GNT4,x.ZNF280D,x.SOX12,x.C21orf62,x.PER3,x.HOXA10,x.HOXC5,x.BLVRB,x.ZIM2,x.HEMK1,x.FAP,x.MAN1A1,x.CDA,x.HTR7P1,x.DALRD3,x.FIBP,x.TTTY15,x.SLC30A1,x.SHANK2,x.MSL2,x.UBOX5,x.DUSP13,x.GJB5,x.MTF2,x.PPP1CA,x.IGHMBP2,x.VEGFA,x.KANSL1L,x.FCN3,x.USP32P2,x.HIVEP3,x.HRH1,x.HDAC7,x.HTT,x.IDH3A,x.TLR3,x.F11R,x.MOAP1,x.ISOC2,x.CLIP3,x.FZD10,x.VOPP1,x.RPL4,x.NUDT2,x.RAB30,x.DBI,x.CCDC87,...,x.ZNHIT3,x.DBF4B,x.GLUD1,x.ADRB2,x.FBXW4,x.SPP1,x.F2R,x.SDF4,x.GALNT8,x.TMEM63A,x.BSCL2,x.GZMM,x.THSD4,x.PRLR,x.PLEKHJ1,x.TLE3,x.PANK2,x.SKP1,x.TOMM70,x.KIAA1324,x.PCDHB12,x.CPA4,x.TRPV2,x.CHCHD2,x.TRPC6,x.MYO1D,x.SLURP1,x.ALG9,x.ZC4H2,x.KIR2DL3,x.IVL,x.BAMBI,x.SLC7A6,x.SLC17A1,x.CCL3,x.PHF8,x.KIR3DL3,x.SARS2,x.PIP4K2C,x.S100A13,x.EPHA1,x.MFGE8,x.OAZ2,x.PCBP3,x.POLA1,x.KREMEN2,x.CYP7B1,x.LILRB3,x.GSAP,y
0,9.83,8.33,5.50,8.69,5.64,6.25,5.84,8.33,5.52,7.66,7.67,8.36,7.34,6.26,5.33,6.19,5.12,6.34,9.63,5.71,7.82,8.06,7.73,8.89,7.20,6.63,9.62,8.66,7.11,8.74,5.46,6.63,9.96,7.93,5.91,8.10,8.39,7.92,5.22,7.70,10.56,8.03,11.72,5.97,9.40,11.96,7.27,6.87,10.73,6.75,...,9.00,5.77,8.48,7.04,8.61,11.07,4.63,10.22,6.43,6.76,9.95,6.44,5.92,4.62,8.45,5.82,8.72,11.46,7.41,6.17,7.33,8.57,6.93,11.70,4.96,5.09,6.52,6.29,8.69,7.27,7.26,6.49,5.59,6.96,6.21,7.37,7.27,7.41,8.99,8.64,7.02,8.44,10.23,8.46,7.35,8.39,4.76,8.23,6.74,0
1,9.63,8.54,5.64,8.83,5.69,6.29,6.07,8.26,5.53,7.57,7.78,8.65,7.28,6.38,5.27,6.20,5.15,6.29,9.94,6.41,8.43,8.34,8.10,8.98,7.14,6.46,9.57,9.05,7.18,9.10,5.41,6.47,10.06,7.93,6.15,8.20,8.18,7.78,5.26,7.78,10.55,8.21,12.05,6.09,9.67,11.75,7.13,6.90,11.55,6.87,...,8.74,5.94,9.03,7.23,8.60,7.01,4.70,10.22,6.51,6.75,10.14,6.63,6.34,4.58,8.64,6.04,8.76,11.30,6.83,6.09,7.35,8.57,6.99,11.85,5.06,5.08,6.77,6.46,8.68,7.30,7.36,6.92,5.60,6.86,7.12,7.35,7.43,7.68,9.21,9.18,7.10,8.63,9.84,8.78,7.48,8.26,4.95,8.33,6.39,0
2,9.69,8.48,5.72,8.50,5.96,6.22,5.77,9.10,5.60,7.42,7.55,8.85,7.24,6.12,5.51,5.99,4.98,6.27,9.81,6.53,8.27,8.58,7.72,8.64,7.08,6.47,9.67,8.64,7.18,8.12,5.54,6.59,10.65,7.89,5.83,8.05,8.14,8.87,5.19,7.40,11.05,7.80,11.79,5.88,9.90,11.82,7.09,6.83,11.43,6.87,...,9.19,5.73,8.67,6.97,8.67,8.66,4.56,10.18,6.33,6.84,10.24,6.53,6.01,4.64,8.59,5.93,8.76,11.61,7.55,5.98,7.08,8.42,6.84,11.68,4.96,5.12,6.44,6.36,8.53,7.14,7.10,7.00,5.54,6.85,5.94,7.30,7.48,7.33,9.27,8.88,6.74,8.19,10.06,8.52,7.30,8.28,4.80,7.98,6.68,0
3,9.99,8.51,5.79,8.42,5.74,6.51,5.85,9.21,5.56,7.33,7.84,8.20,7.21,6.28,5.33,5.82,4.86,6.64,9.75,6.44,8.62,8.59,7.92,8.69,6.99,6.37,9.48,8.81,7.24,8.82,5.47,6.65,10.29,7.72,5.79,8.36,8.22,8.01,5.28,7.42,10.71,7.98,12.00,6.01,9.85,11.88,7.18,6.58,12.04,6.65,...,9.06,5.62,8.94,6.85,8.89,8.10,4.56,10.43,6.36,6.56,10.27,6.34,6.00,4.67,8.40,5.71,8.80,11.44,7.11,5.84,7.17,8.18,6.72,11.72,4.93,5.15,6.28,6.54,8.25,7.06,7.06,6.80,5.48,6.74,6.04,7.38,7.18,7.31,9.02,8.71,6.85,8.28,9.91,8.21,7.45,8.28,4.96,7.89,6.82,0
4,9.58,8.37,5.78,8.82,5.63,6.20,5.97,8.50,5.60,7.54,7.60,8.58,7.53,6.34,5.33,6.02,5.11,6.12,9.58,5.79,8.30,8.61,7.55,8.83,7.10,6.26,9.52,8.61,7.17,9.14,5.40,6.73,9.79,7.86,5.91,7.80,8.34,7.99,5.23,7.25,10.72,7.96,11.78,5.73,9.33,11.93,7.20,6.77,11.44,6.70,...,8.91,5.44,8.84,6.79,8.64,9.43,4.70,10.09,6.35,6.83,10.11,6.68,6.25,4.57,8.66,5.81,8.70,11.30,7.53,5.99,7.26,8.80,6.65,11.46,5.07,5.14,6.59,6.32,8.72,7.37,7.47,6.54,5.53,6.92,6.10,7.24,7.25,7.56,9.23,8.85,6.84,8.04,10.03,8.76,7.25,8.47,4.87,8.13,7.00,0


In [21]:
df2.y.value_counts()

4    39
0    38
1    34
3    31
5    26
2    15
6     6
Name: y, dtype: int64

In [22]:
X = df2.iloc[:,0:500]
y = df2.iloc[:,500]

In [23]:
X.values, y.values

(array([[ 9.82567961,  8.3271627 ,  5.4993819 , ...,  4.76001706,
          8.23018205,  6.74038468],
        [ 9.63124675,  8.54282676,  5.64429229, ...,  4.95237261,
          8.33250157,  6.39068256],
        [ 9.69054774,  8.47648577,  5.71718694, ...,  4.79623236,
          7.97819429,  6.67703781],
        ...,
        [ 9.3618059 ,  8.26365314, 10.29656824, ...,  4.88436628,
          8.1768516 ,  5.61143535],
        [ 9.3460355 ,  8.71853424,  9.66081321, ...,  4.9512614 ,
          8.63041198,  5.79046804],
        [ 9.44789324,  8.25538598,  8.46580862, ...,  4.99628241,
          8.70066461,  5.17747408]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
   

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.5, random_state=0)

In [25]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((94, 500), (95, 500), (94,), (95,))

In [26]:
knn_r_acc = []

for i in range(1,12,2):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    accscore = accuracy_score(y_test,y_pred)
    knn_r_acc.append((i,accscore))

In [27]:
result = pd.DataFrame(knn_r_acc, columns=['K','Accuracy Score'])
result

,K,Accuracy Score
0,1,0.98
1,3,0.98
2,5,0.94
3,7,0.95
4,9,0.94
5,11,0.92


#### Python code done by Dennis Lam